### Importing Libraries

In [1]:
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
)
from azure.identity import DefaultAzureCredential

### Workspace Configuration

In [2]:
subscription_id = ""
resource_group = "product_detection"
workspace = "az_deployment"

In [3]:
ml_client = MLClient(
    DefaultAzureCredential(),
    subscription_id = subscription_id,
    resource_group = resource_group,
    workspace = workspace
)

### Endpoint Configuration

In [4]:
import datetime
endpoint_name = "endpt-mdl1"+ datetime.datetime.now().strftime("%m%d%H%M%f")
endpoint = ManagedOnlineEndpoint(
    name = endpoint_name,
    description = "Test Endpoint",
    auth_mode = "key"
)

### Deployment Configuration

In [10]:
model = Model(path="best.pt")
env = Environment(
    conda_file="conda.yaml",
    image="umairsiddique3171/azlocaldeployment:lastest"
)
test_deployment = ManagedOnlineDeployment(
    name="test",
    endpoint_name=endpoint_name,
    model=model,
    environment=env,
    code_configuration=CodeConfiguration(
        code=".", scoring_script="score.py"
    ),
    instance_type="Standard_DS3_v2",
    instance_count=1,
)

### Local Deployment

In [11]:
ml_client.online_endpoints.begin_create_or_update(endpoint=endpoint,local=True)     ### local docker deployment for final testing

Updating local endpoint (endpt-mdl110021315243558) .

Done (0m 5s)


ManagedOnlineEndpoint({'public_network_access': None, 'provisioning_state': None, 'scoring_uri': None, 'openapi_uri': None, 'name': 'endpt-mdl110021315243558', 'description': 'Test Endpoint', 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': None, 'Resource__source_path': '', 'base_path': WindowsPath('C:/Users/US593/.azureml/inferencing/endpt-mdl110021315243558'), 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x000002C46AE4F400>, 'auth_mode': 'key', 'location': None, 'identity': None, 'traffic': {}, 'mirror_traffic': {}, 'kind': None})

In [12]:
ml_client.online_deployments.begin_create_or_update(deployment=test_deployment,local=True)   # local docker deployment for final testing

Creating local deployment (endpt-mdl110021315243558 / test) .
Building Docker image from Dockerfile
Step 1/6 : FROM umairsiddique3171/azlocaldeployment:lastest
 ---> 8ebf443213a4
Step 2/6 : RUN mkdir -p /var/azureml-app/
 ---> Running in d04e361f3710
 ---> 805be950171c
Step 3/6 : WORKDIR /var/azureml-app/
 ---> Running in 6fc86282efc2
 ---> 1a406af02b73
Step 4/6 : COPY conda.yml /var/azureml-app/
 ---> 9a362d5b7b1e
Step 5/6 : RUN conda env create -n inf-conda-env --file conda.yml
 ---> Running in 513424a3d0a9
.Retrieving notices: ...working... done
Channels:
 - defaults
 - conda-forge
Platform: linux-64
Solving environment: ...working... done


==> WARNING: A newer version of conda exists. <==
    current version: 24.7.1
    latest version: 24.9.1

Please update conda by running

    $ conda update -n base -c conda-forge conda


.....
Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done
Installing pip dependencies

ManagedOnlineDeployment({'private_network_connection': None, 'package_model': False, 'provisioning_state': 'Succeeded', 'endpoint_name': 'endpt-mdl110021315243558', 'type': 'Managed', 'name': 'test', 'description': None, 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': None, 'Resource__source_path': '', 'base_path': WindowsPath('c:/Users/US593/OneDrive/Desktop/az-local-deployment/src'), 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x000002C46ADB0580>, 'model': Model({'job_name': None, 'intellectual_property': None, 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': '51fcbd004418bb09a0521df13f902646', 'description': None, 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': None, 'Resource__source_path': '', 'base_path': WindowsPath('c:/Users/US593/OneDrive/Desktop/az-local-deployment/src'), 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x000002C46B256430>

### Local Deployment Status

In [14]:
ml_client.online_endpoints.get(name=endpoint_name,local=True)

ManagedOnlineEndpoint({'public_network_access': None, 'provisioning_state': 'Succeeded', 'scoring_uri': 'http://localhost:32770/score', 'openapi_uri': None, 'name': 'endpt-mdl110021315243558', 'description': 'Test Endpoint', 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': None, 'Resource__source_path': '', 'base_path': WindowsPath('c:/Users/US593/OneDrive/Desktop/az-local-deployment/src'), 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x000002C46B263850>, 'auth_mode': 'key', 'location': 'local', 'identity': None, 'traffic': {}, 'mirror_traffic': {}, 'kind': None})

### Scoring Local Endpoint

In [15]:
import requests
import json
import base64
from PIL import Image

endpoint_url = "http://localhost:32770/score"

def load_image(image_path):
    with open(image_path, "rb") as img_file:
        img_bytes = img_file.read()
    return base64.b64encode(img_bytes).decode("utf-8")

def invoke_local_endpoint(image_path):
    base64_image = load_image(image_path)
    payload = json.dumps({
        "data": base64_image
    })
    headers = {
        'Content-Type': 'application/json'
    }

    try:
        response = requests.post(endpoint_url, headers=headers, data=payload)

        if response.status_code == 200:
            result = response.json()
            return result
        else:
            print(f"Error: {response.status_code} - {response.text}")
    except Exception as e:
        print(f"Failed to invoke endpoint: {str(e)}")

image_path = "test.jpg"
predictions = invoke_local_endpoint(image_path)
print(predictions)


{"predictions": [{"box": [2307.440185546875, 1102.9967041015625, 2730.507080078125, 1611.138671875], "confidence": 0.91904217004776, "class": "Promo_price"}, {"box": [2303.176025390625, 2318.26318359375, 2633.460693359375, 2573.544677734375], "confidence": 0.8977777361869812, "class": "Regular_price"}, {"box": [3206.9091796875, 1109.337158203125, 3639.017333984375, 1626.1539306640625], "confidence": 0.8913969993591309, "class": "Promo_price"}, {"box": [3347.99072265625, 2.0687015056610107, 3779.1748046875, 305.62664794921875], "confidence": 0.8871928453445435, "class": "Regular_price"}, {"box": [1732.6656494140625, 2.4472389221191406, 2157.72412109375, 297.1795959472656], "confidence": 0.8800140619277954, "class": "Regular_price"}, {"box": [2208.193359375, 2.4831676483154297, 2648.618896484375, 319.08685302734375], "confidence": 0.8762866258621216, "class": "Regular_price"}, {"box": [1731.0587158203125, 1103.1456298828125, 2153.1044921875, 1614.0447998046875], "confidence": 0.869373381